In [1]:
from pysmt.shortcuts import Not, And, Or, Symbol, FALSE
from pysmt.shortcuts import GE, LE, LT, GT, Equals, Plus, Times, Real
from pysmt.shortcuts import Solver, Real
from pysmt.typing import REAL, INT

import numpy as np

In [2]:
from sage.all import Polyhedron

In [3]:
def pysmt_to_sage(f):
    '''
    Convert a formula into its corresponding polyhedron in Sage

    :param f: fNode 
    :return: Polyhedron
    '''

    pass


def sage_to_pysmt(P):
    '''
    Convert a polyhedron in Sage into its corresponding formula for pysmt

    :param P: Polyhedron
    :return: fNode
    '''

    pass

In [4]:
def inv(l):
    '''
    Invert each element of a tuple

    :param l: a tuple
    :return: a tuple
    '''

    return [-i for i in list(l)]


def H_to_array(P):
    '''
    Convert a polyhedron  P into two arrays A,b
    such that P = {x | Ax >= b}

    :param P: Polyhedron()
    :return: (A=np.array,b=np.array)
    '''

    H =  P.Hrepresentation()
    A = []
    b = []
    for ieq in H:
        if ieq.is_inequality():
            A.append(list(ieq.A()))
            b.append(ieq.b())
        elif ieq.is_equation():
            inv(list(ieq.A()))
            A.append(list(ieq.A()))
            A.append(inv(list(ieq.A())))
            b.append(ieq.b())
            b.append(-ieq.b())
    return (np.array(A),np.array(b))


def array_to_H(A,b):
    '''
    Convert two arrays A,b into a polyhedron P
    such that P = {x | Ax >= b}

    :param A: np.array
    :param b: np.array
    :return: Polyhedron()
    '''

    assert(len(A)==len(b))
    return Polyhedron(ieqs=[[b[i]] + list(A[i]) for i in range(len(A))])

In [5]:
x = Symbol("x",REAL)
y = Symbol("y",REAL)


# formula = Not(And(Not(LE(x,Real(3))), GE(y,Real(1)), LE(y,Real(2)), Or(GE(y,Real(3)), LE(y,Real(2)))))
formula = Not(And(Not(LE(x,Real(3))), Not(LT(y,Real(1))), LE(y,Real(2)), Or(Not(LT(y,Real(3))), LT(y,Real(2)))))
formula.serialize()

'(! ((! (x <= 3.0)) & (! (y < 1.0)) & (y <= 2.0) & ((! (y < 3.0)) | (y < 2.0))))'

In [6]:
def get_subsolution(formula,variables):
    '''
    Give values for a subset of variables that satisfies the formula

    :param formula: FNode
    :param variables: list of Symbol
    :return: None if the formula is not sat and a list of Real for each variable
    '''

    model = []
    with Solver() as solver:
        solver.add_assertion(formula)
        solver.solve()
        if solver.solve():
            for v in variables:
                model.append(solver.get_value(v))
            return model
        else:
            return None


get_subsolution(formula,[x]), get_subsolution(formula,[y])

([4.0], [5/2])

In [7]:
list(formula.args())[0].args(), list(formula.args())[0].is_and(), list(formula.args())[0].is_or()

(((! (x <= 3.0)), (! (y < 1.0)), (y <= 2.0), ((! (y < 3.0)) | (y < 2.0))),
 True,
 False)

In [8]:
def nnf(formula):
    '''
    Compute the negative normal form of a formula

    :param formula: fNode
    :return: fNode
    '''

    if formula.is_and():
        return And([nnf(sf) for sf in formula.args()])
    elif formula.is_or():
        return Or([nnf(sf) for sf in formula.args()])
    elif formula.is_not():
        nf = list(formula.args())[0]
        if nf.is_and():
            return Or([nnf(Not(sf)) for sf in nf.args()])
        elif nf.is_or():
            return And([nnf(Not(sf)) for sf in nf.args()])
        elif nf.is_not():
            return nnf(list(nf.args())[0])
        elif nf.is_le() or nf.is_lt():
            return formula
        else:
            print("Error no operator found")
            assert(False)
    else:
        return formula



print("formula:",formula)
print("nnf:",nnf(formula))

formula: (! ((! (x <= 3.0)) & (! (y < 1.0)) & (y <= 2.0) & ((! (... < ...)) | (y < 2.0))))
nnf: ((x <= 3.0) | (y < 1.0) | (! (y <= 2.0)) | ((y < 3.0) & (! (y < 2.0))))


In [9]:
def check(formula, variables=[], model=[]):
    '''
    Model-check and sat a formula for a subset of variables

    :param formula: fNode
    :param variables: list of Symbol
    :param model: list of Real
    :return: bool
    '''

    assert(len(variables) == len(model))
    with Solver() as solver:
        solver.add_assertion(formula)
        for i in range(len(variables)):
            solver.add_assertion(Equals(variables[i],model[i]))
        return solver.solve()

def get_true_atoms(formula, variables, p):
    '''
    Compute the atoms that p satisfies in the formula

    :param formula: fNode
    :param variables: list of Symbol
    :param p: list of Real
    :return: list of fNode
    '''

    if formula.is_and():
        atoms = []
        for sf in formula.args():
            rec = get_true_atoms(sf, variables, p)
            if rec == []:
                return []
            else:
                atoms += rec
        return atoms
    elif formula.is_or():
        for sf in formula.args():
            atoms = get_true_atoms(sf, variables, p)
            if atoms != []:
                return atoms
        return []
    elif formula.is_le() or formula.is_lt():
        if check(formula, variables, p):
            return [formula]
        else:
            return []
    elif formula.is_not():
        nf = formula.args()[0]
        if nf.is_le() or nf.is_lt():
            if check(formula, variables, p):
                return [formula]
            else:
                return []
        else:
            print("Error, formula must be in nnf")
            assert(False)
    else:
        print("Error no operator found")
        assert(False)

formula = And(LT(x,Real(0)), Or(Not(LE(x,Real(2))), Not(LE(y,Real(2))), LT(x,Real(2))), Or(LT(y,Real(2)), LE(y,Real(2))))
formula = nnf(formula)
variables = [x,y]
print(formula, variables)
print(get_subsolution(formula,variables))
get_true_atoms(formula, variables, get_subsolution(formula,variables))

((x < 0.0) & ((! (x <= 2.0)) | (! (y <= 2.0)) | (x < 2.0)) & ((y < 2.0) | (y <= 2.0))) [x, y]
[-1.0, 0.0]


[(x < 0.0), (x < 2.0), (y < 2.0)]

In [10]:
def ddnf(formula, variables):
    '''
    Compute a set of disjoint polyedra equivalent to the formula

    :param formula: fNode
    :param variables: list of Symbol
    :return: list of Polyhedron
    '''

    poly = []
    while check(formula):
        f = nnf(formula)
        atoms = get_true_atoms(f, variables, get_subsolution(f,variables))
        poly.append(atoms)
        formula = And(formula, (Not(And(atoms))))
    return poly


formula = And(Or(And(LE(x,Real(0)), Not(LT(y,Real(1)))),
                 And(LT(x,Real(-2)), LT(y,Real(1)))),
              And(LE(x, Real(0)), Not(LE(x, Real(-5))),LE(y, Real(5)), Not(LE(y, Real(-5)))))
print(formula.serialize())
ddnf(formula,variables)


((((x <= 0.0) & (! (y < 1.0))) | ((x < -2.0) & (y < 1.0))) & ((x <= 0.0) & (! (x <= -5.0)) & (y <= 5.0) & (! (y <= -5.0))))


[[(x < -2.0),
  (y < 1.0),
  (x <= 0.0),
  (! (x <= -5.0)),
  (y <= 5.0),
  (! (y <= -5.0))],
 [(x <= 0.0),
  (! (y < 1.0)),
  (x <= 0.0),
  (! (x <= -5.0)),
  (y <= 5.0),
  (! (y <= -5.0)),
  (! (y < 1.0))]]

In [11]:
def minimal_faces(P):
    '''
    Compute a representation of the minimal faces of a polyhedron P
    :param P: Polyhedron
    :return: list of (p=array, l=list of array) with p a point and l a basis of a span
    '''

    return [[p.vector(), [v.vector() for v in P.lines()]] for p in P.vertices()]

P_full_space = Polyhedron(vertices=[[9,41]],lines=[[41,-3],[7,7]])
P_half_space = Polyhedron(ieqs=[[1,1,0]])
P_line = Polyhedron(eqns=[[1,1,1]])
P_half_line = Polyhedron(ieqs=[[1,1,0]], eqns=[[-1,0,1]])
P_polytope = Polyhedron(vertices=[[1,5,3],[4,8,7],[0,0,0],[-5,1,4]])

print("minimal faces:")
print("P_full_space:",minimal_faces(P_full_space))
print("P_half_space:",minimal_faces(P_half_space))
print("P_line:",minimal_faces(P_line))
print("P_half_line:",minimal_faces(P_half_line))
print("P_polytope:",minimal_faces(P_polytope))

minimal faces:
P_full_space: [[(0, 0), [(0, 1), (1, 0)]]]
P_half_space: [[(-1, 0), [(0, 1)]]]
P_line: [[(0, -1), [(1, -1)]]]
P_half_line: [[(-1, 1), []]]
P_polytope: [[(-5, 1, 4), []], [(0, 0, 0), []], [(1, 5, 3), []], [(4, 8, 7), []]]


In [12]:
def min_matrices_faces(P,M,c):
    '''
    Compute the list matrix D associated to each minimal face of the polyhedron F = {(D,E)x = f}

    :param P: Polyhedron
    :param M,c: arrays such that P={Mx <= c}
    :return: list of Polyhedron
    '''

    min_faces = minimal_faces(P)

    l_D = []
    for p,_ in min_faces:
        D = []
        for i in range(len(M)):
            l = M[i]
            if np.dot(l,np.array(p)) == c[i]:
                D.append(l)
        l_D.append(np.array(D))
    return l_D

def qe_polyhedron(P,vx):
    '''
    Eliminate the variables -which index is not in vx- in the definition of P and returns the equivalent list of polyhedron

    :param P: Polyhedron
    :param vx: list of int
    :return: list of Polyhedron
    '''

    M,c = H_to_array(P)
    A,B = [],[]
    for l in range(len(M)):
        if l in vx:
            A.append(M.T[l])
        else:
            B.append(M.T[l])
    A = np.array(A).T
    B = -np.array(B).T

    # Ax <= By +c

    l_D = min_matrices_faces(P,M,c)

    # U_D A(D(By+c)) <= By+c


    return "[array_to_H(A.dot(D.dot(B)) - B, c - A.dot(D.dot(c))) for D in l_D]"

qe_polyhedron(P_line,[])

'[array_to_H(A.dot(D.dot(B)) - B, c - A.dot(D.dot(c))) for D in l_D]'

In [13]:
def poly_valid(P):
    '''
    Check if the polyhedron is equivalent to TRUE

    :param P: Polyhedron
    :return: None if P is equivalent to TRUE and y a conter-example otherwise 
    '''

    with Solver as solver:
        solver.add_assertion(sage_to_pysmt(P))

        if solver.solve():
            return solver.get_value()
        else:
            return None

In [16]:
variables = [x,y]
formula = And(LT(x,Real(0)),LE(x,y)) #forall y, exists x formula
list_ddnf = ddnf(formula,variables)

v_quant = [y]
v_elim = [x]

is_formula_true = False


for P in list_ddnf:
    list_elim = qe_polyhedron(P,v_elim)
    for P_elim in list_elim:
        vy = poly_valid(P_elim)
        if vy == None:
            is_formula_true = True
        else:
            if not check(formula, v_quant, vy):
                is_formula_true = False
            else:
                formula = And(formula)